# 2018-2 spark Project

---

## RDD

5개 이상 메소드

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import pandas as pd

### train data 읽기

In [3]:
act = sc.textFile('file:///home/ubuntu/18-2E/sparkProject/train_activity.csv').map(lambda line : line.split(','))
label = sc.textFile('file:///home/ubuntu/18-2E/sparkProject/train_label.csv').map(lambda line : line.split(','))

### pandas로 확인하기

In [4]:
pd_act = pd.DataFrame(act.collect())
pd_act.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,wk,acc_id,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,...,cnt_clear_bam,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt
1,7,3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd...,4,2.08881416107027,4.4050571352657,-0.2454767368248,4.45405933933072,3.0097968979651,-0.306353601768793,1.18401086299851,...,-0.0331304236986033,-0.0340376613356785,-0.23182695053692,-0.0165738550449223,-0.275173788641312,-0.343293159909132,-0.0240280218289566,3.46693878868001,-0.0277583111202987,-0.365536464552178
2,8,3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd...,5,2.67346049372266,4.76017781944869,-0.222526822840016,6.10747941074093,4.75852847138303,-0.306353601768793,1.70953391447875,...,-0.0331304236986033,-0.0340376613356785,-0.23182695053692,-0.0165738550449223,-0.275173788641312,-0.343293159909132,-0.0240280218289566,5.07035860175402,-0.0277583111202987,-0.365536464552178
3,3,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,2,-0.649521652982493,-0.231020534896592,-0.290803194162271,-0.245977789538834,-0.487666248870767,-0.306353601768793,-0.566706303287821,...,-0.0331304236986033,-0.0340376613356785,-0.23182695053692,-0.0165738550449223,-0.275173788641312,-0.343293159909132,-0.0240280218289566,-0.498275073381396,-0.0277583111202987,-0.365536464552178
4,4,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,2,-0.65823531742848,-0.231874043837118,-0.290803194162271,-0.250423355452758,-0.487666248870767,-0.306353601768793,-0.569841814936623,...,-0.0331304236986033,-0.0340376613356785,-0.23182695053692,-0.0165738550449223,-0.275173788641312,-0.343293159909132,-0.0240280218289566,-0.503692032209349,-0.0277583111202987,-0.365536464552178


In [5]:
pd_label = pd.DataFrame(label.collect())
pd_label.head()

,0,1
0,acc_id,label
1,b8fbf3f6a70e3f36843bffc70c18ff51a0d755a87616ec...,week
2,ed500c4957956b3e99dc3985666850b582f812405eefb6...,week
3,acc6afa23a6bf15e18151e4794c7789225ef9d682f473c...,week
4,34095a3c9a2937ced3ea3fd75e22ce177dc5879d2a53f7...,week


In [6]:
# column 정보 제거
act = act.filter(lambda line : line[0] != "wk")
label = label.filter(lambda line: line[0] != "acc_id")

### 각 주차에 한번이라도 활동한 사람 수

wk : 활동 주 (1~8)

In [7]:
result1 = act.groupBy(lambda x : x[0])\
    .mapValues(lambda x : len(x))\
    .sortBy(lambda x: x[1], False)
result1.collect()

[('8', 100000),
 ('7', 62838),
 ('5', 52585),
 ('4', 51430),
 ('6', 50548),
 ('3', 46122),
 ('2', 43093),
 ('1', 33707)]

시간이 지남에 따라 8주차에 가까워 질수록 활동 인원이 증가한 것을 확인하였다

### 각 주차별 모든 사람의 접속일수 합

cnt_dt : 해당 주에 한번 이상 접속한 일수

In [8]:
result2 = act.map(lambda x : (int(x[0]), int(x[2])))
result2.take(10)

[(7, 4),
 (8, 5),
 (3, 2),
 (4, 2),
 (5, 4),
 (7, 2),
 (8, 5),
 (8, 6),
 (8, 3),
 (1, 6)]

(주차, 접속일수) pair들을 만들어서 각 주차별로 총 접속일수를 reduceByKey 메소드를 이용하여 더해보았다.

In [9]:
result2 = act.map(lambda x : (int(x[0]), int(x[2])))\
    .reduceByKey(lambda a, b : a+b)\
    .sortBy(lambda x : x[1], False)
result2.collect()

[(8, 405036),
 (7, 282452),
 (5, 241248),
 (4, 235921),
 (3, 221524),
 (6, 214319),
 (2, 181008),
 (1, 165063)]

6주차의 경우 접속빈도는 조금 낮은것을 볼 수 있었다

---

## SparkSQL

7개 이상 메소드

In [10]:
source = "file:///home/ubuntu/18-2E/sparkProject/train_activity.csv"
df = spark.read.csv(source, header = "true")

header 옵션을 true로 해주어 맨 윗 헤더값들을 column으로 넣어주었다

In [11]:
df.show(3)

+---+--------------------+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+---------------------+----------------------+------------------------+-----------------------+------------------+--------------------+-------------------+---------------------+----------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+-----------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+
| wk|              acc_id|cnt_dt|         play_time|           npc_exp|       npc_hongmun|         quest_exp|     quest_hongmun|      item_hongmun|  game_combat_time|          get_money|           duel_cnt|           duel_win|   partybattle_cnt|   party

In [12]:
df.head()

Row(wk='7', acc_id='3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd109415062b2bd58709', cnt_dt='4', play_time='2.08881416107027', npc_exp='4.4050571352657', npc_hongmun='-0.2454767368248', quest_exp='4.45405933933072', quest_hongmun='3.0097968979651', item_hongmun='-0.306353601768793', game_combat_time='1.18401086299851', get_money='-0.023442516280297', duel_cnt='-0.0361680430708607', duel_win='-0.0413257061219071', partybattle_cnt='-0.167472539558229', partybattle_win='-0.159872456051349', cnt_enter_inzone_solo='-0.308749014788707', cnt_enter_inzone_light='-0.424591467385917', cnt_enter_inzone_skilled='-0.190671429600539', cnt_enter_inzone_normal='-0.325565585196072', cnt_enter_raid='-0.368129648987356', cnt_enter_raid_light='-0.402052757250742', cnt_enter_bam='-0.0476429005602935', cnt_clear_inzone_solo='-0.306502306156819', cnt_clear_inzone_light='-0.430263339890395', cnt_clear_inzone_skilled='-0.188787865618004', cnt_clear_inzone_normal='-0.309353682254944', cnt_clear_raid='-0.36761

In [13]:
df.take(2)

[Row(wk='7', acc_id='3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd109415062b2bd58709', cnt_dt='4', play_time='2.08881416107027', npc_exp='4.4050571352657', npc_hongmun='-0.2454767368248', quest_exp='4.45405933933072', quest_hongmun='3.0097968979651', item_hongmun='-0.306353601768793', game_combat_time='1.18401086299851', get_money='-0.023442516280297', duel_cnt='-0.0361680430708607', duel_win='-0.0413257061219071', partybattle_cnt='-0.167472539558229', partybattle_win='-0.159872456051349', cnt_enter_inzone_solo='-0.308749014788707', cnt_enter_inzone_light='-0.424591467385917', cnt_enter_inzone_skilled='-0.190671429600539', cnt_enter_inzone_normal='-0.325565585196072', cnt_enter_raid='-0.368129648987356', cnt_enter_raid_light='-0.402052757250742', cnt_enter_bam='-0.0476429005602935', cnt_clear_inzone_solo='-0.306502306156819', cnt_clear_inzone_light='-0.430263339890395', cnt_clear_inzone_skilled='-0.188787865618004', cnt_clear_inzone_normal='-0.309353682254944', cnt_clear_raid='-0.3676

In [14]:
df.describe("wk").show()

+-------+-----------------+
|summary|               wk|
+-------+-----------------+
|  count|           440323|
|   mean|5.155438166981965|
| stddev| 2.31232206450818|
|    min|                1|
|    max|                8|
+-------+-----------------+



In [15]:
df.printSchema()

root
 |-- wk: string (nullable = true)
 |-- acc_id: string (nullable = true)
 |-- cnt_dt: string (nullable = true)
 |-- play_time: string (nullable = true)
 |-- npc_exp: string (nullable = true)
 |-- npc_hongmun: string (nullable = true)
 |-- quest_exp: string (nullable = true)
 |-- quest_hongmun: string (nullable = true)
 |-- item_hongmun: string (nullable = true)
 |-- game_combat_time: string (nullable = true)
 |-- get_money: string (nullable = true)
 |-- duel_cnt: string (nullable = true)
 |-- duel_win: string (nullable = true)
 |-- partybattle_cnt: string (nullable = true)
 |-- partybattle_win: string (nullable = true)
 |-- cnt_enter_inzone_solo: string (nullable = true)
 |-- cnt_enter_inzone_light: string (nullable = true)
 |-- cnt_enter_inzone_skilled: string (nullable = true)
 |-- cnt_enter_inzone_normal: string (nullable = true)
 |-- cnt_enter_raid: string (nullable = true)
 |-- cnt_enter_raid_light: string (nullable = true)
 |-- cnt_enter_bam: string (nullable = true)
 |-- cnt

#### sql문을 직접 이용한 연산

In [16]:
df.createOrReplaceTempView("act")
spark.sql("SELECT wk, acc_id, cnt_dt FROM act WHERE wk > 6").show(5)

+---+--------------------+------+
| wk|              acc_id|cnt_dt|
+---+--------------------+------+
|  7|3dc6f2875dc6e6f35...|     4|
|  8|3dc6f2875dc6e6f35...|     5|
|  7|b8856358ff62e596f...|     2|
|  8|b8856358ff62e596f...|     5|
|  8|fa883ca7505082114...|     6|
+---+--------------------+------+
only showing top 5 rows



주차가 6보다 큰 경우, wk, acc_id, cnt_dt 출력

In [17]:
heavy = spark.sparkContext.broadcast([5,6,7])
df.select(df['acc_id'],df['wk'],df['cnt_dt'], df['cnt_dt'].isin(heavy.value).alias("heavyUser")).show(10)

+--------------------+---+------+---------+
|              acc_id| wk|cnt_dt|heavyUser|
+--------------------+---+------+---------+
|3dc6f2875dc6e6f35...|  7|     4|    false|
|3dc6f2875dc6e6f35...|  8|     5|     true|
|b8856358ff62e596f...|  3|     2|    false|
|b8856358ff62e596f...|  4|     2|    false|
|b8856358ff62e596f...|  5|     4|    false|
|b8856358ff62e596f...|  7|     2|    false|
|b8856358ff62e596f...|  8|     5|     true|
|fa883ca7505082114...|  8|     6|     true|
|d094b6b1c5d0a147e...|  8|     3|    false|
|38e7088d64485baba...|  1|     6|     true|
+--------------------+---+------+---------+
only showing top 10 rows



각 사람이 그 주차에 5번 이상 게임에서 활동했다면, heavyUser!

In [18]:
from pyspark.sql import functions

col = functions.when(df.cnt_dt >= 5 , "heavy").otherwise("light").alias("userType")
df.select(df.acc_id, df.wk, df.cnt_dt, col).show(10)

+--------------------+---+------+--------+
|              acc_id| wk|cnt_dt|userType|
+--------------------+---+------+--------+
|3dc6f2875dc6e6f35...|  7|     4|   light|
|3dc6f2875dc6e6f35...|  8|     5|   heavy|
|b8856358ff62e596f...|  3|     2|   light|
|b8856358ff62e596f...|  4|     2|   light|
|b8856358ff62e596f...|  5|     4|   light|
|b8856358ff62e596f...|  7|     2|   light|
|b8856358ff62e596f...|  8|     5|   heavy|
|fa883ca7505082114...|  8|     6|   heavy|
|d094b6b1c5d0a147e...|  8|     3|   light|
|38e7088d64485baba...|  1|     6|   heavy|
+--------------------+---+------+--------+
only showing top 10 rows



In [19]:
df.select(functions.mean(df['cnt_dt'])).show()

+-----------------+
|      avg(cnt_dt)|
+-----------------+
|4.420779745777532|
+-----------------+



평균 일주일 활동 일 수 

In [20]:
df.select(functions.collect_set('wk')).show()

+--------------------+
|     collect_set(wk)|
+--------------------+
|[3, 1, 2, 5, 8, 4...|
+--------------------+



주차는 8주차까지~

In [21]:
df.select(functions.countDistinct('wk')).show()

+------------------+
|count(DISTINCT wk)|
+------------------+
|                 8|
+------------------+



주차는 8주차까지~2

---

## SparkML

### Decision Tree

In [35]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.pipeline import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql import functions

In [36]:
# 1. 스파크세션 생성
spark = SparkSession \
    .builder \
    .appName("decision_tree_userType") \
    .master("local[*]") \
    .getOrCreate()

In [37]:
def isHeavyUser(cnt_dt):
    if cnt_dt >= 5:
        return 1.0
    else:
        return 0.0

In [38]:
# Label(heavyUser:1.0, lightUser:0.0)
isHU = functions.udf(lambda cnt_dt: isHeavyUser(cnt_dt))

In [39]:
d1 = spark.read.option("header", "true") \
    .option("sep", ",").option("inferSchema", True) \
    .option("mode", "DROPMALFORMED") \
    .csv("file:///home/ubuntu/18-2E/sparkProject/train_activity.csv")

d2 = d1.toDF('wk', 'acc_id', 'cnt_dt', 'play_time', 'npc_exp', 'npc_hongmun',
       'quest_exp', 'quest_hongmun', 'item_hongmun', 'game_combat_time',
       'get_money', 'duel_cnt', 'duel_win', 'partybattle_cnt',
       'partybattle_win', 'cnt_enter_inzone_solo', 'cnt_enter_inzone_light',
       'cnt_enter_inzone_skilled', 'cnt_enter_inzone_normal', 'cnt_enter_raid',
       'cnt_enter_raid_light', 'cnt_enter_bam', 'cnt_clear_inzone_solo',
       'cnt_clear_inzone_light', 'cnt_clear_inzone_skilled',
       'cnt_clear_inzone_normal', 'cnt_clear_raid', 'cnt_clear_raid_light',
       'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat',
       'party_chat', 'guild_chat', 'faction_chat', 'cnt_use_buffitem',
       'gathering_cnt', 'making_cnt')

d2.printSchema()

root
 |-- wk: integer (nullable = true)
 |-- acc_id: string (nullable = true)
 |-- cnt_dt: integer (nullable = true)
 |-- play_time: double (nullable = true)
 |-- npc_exp: double (nullable = true)
 |-- npc_hongmun: double (nullable = true)
 |-- quest_exp: double (nullable = true)
 |-- quest_hongmun: double (nullable = true)
 |-- item_hongmun: double (nullable = true)
 |-- game_combat_time: double (nullable = true)
 |-- get_money: double (nullable = true)
 |-- duel_cnt: double (nullable = true)
 |-- duel_win: double (nullable = true)
 |-- partybattle_cnt: double (nullable = true)
 |-- partybattle_win: double (nullable = true)
 |-- cnt_enter_inzone_solo: double (nullable = true)
 |-- cnt_enter_inzone_light: double (nullable = true)
 |-- cnt_enter_inzone_skilled: double (nullable = true)
 |-- cnt_enter_inzone_normal: double (nullable = true)
 |-- cnt_enter_raid: double (nullable = true)
 |-- cnt_enter_raid_light: double (nullable = true)
 |-- cnt_enter_bam: double (nullable = true)
 |-- c

In [40]:
# 사람별 평균 cnt_dt (주당 방문 일수)
d3 = d2.groupBy("acc_id").agg(functions.round(functions.avg("cnt_dt"), 1).alias("mean(cnt_dt)"))
d4 = d2.join(d3, ["acc_id"])
d4.select(d4["acc_id"], d4["mean(cnt_dt)"]).show(20, False)

+----------------------------------------------------------------+------------+
|acc_id                                                          |mean(cnt_dt)|
+----------------------------------------------------------------+------------+
|00446675fab526fc7b768e18ed051e3b5e341d5078fd2508c9c03f5258a2389a|3.7         |
|00446675fab526fc7b768e18ed051e3b5e341d5078fd2508c9c03f5258a2389a|3.7         |
|00446675fab526fc7b768e18ed051e3b5e341d5078fd2508c9c03f5258a2389a|3.7         |
|0148a24b0c6ea3da5f03ac5f516fe030d63fb88d222f1cd417073c7bb7edd71e|1.0         |
|02a4d8afc1c0359a3c0d28e3cd55cd8956ba02af055260a50c113b91e91e4573|2.0         |
|02a4d8afc1c0359a3c0d28e3cd55cd8956ba02af055260a50c113b91e91e4573|2.0         |
|033611b3c479b8d62fb04b2377378f46d8017170ef758ce3f594b04934de7f4d|6.5         |
|033611b3c479b8d62fb04b2377378f46d8017170ef758ce3f594b04934de7f4d|6.5         |
|033611b3c479b8d62fb04b2377378f46d8017170ef758ce3f594b04934de7f4d|6.5         |
|033611b3c479b8d62fb04b2377378f46d801717

한 사람이 여러번 나오기는 하지만 각 사람별로 평균 cnt_dt 값을 확인할 수 있다

In [41]:
# label 부여
d5 = d4.withColumn("isHeavyUser", isHU(d4["mean(cnt_dt)"]).cast("double"))
d5.select("acc_id", "mean(cnt_dt)", "isHeavyUser").show(20, False)

+----------------------------------------------------------------+------------+-----------+
|acc_id                                                          |mean(cnt_dt)|isHeavyUser|
+----------------------------------------------------------------+------------+-----------+
|00446675fab526fc7b768e18ed051e3b5e341d5078fd2508c9c03f5258a2389a|3.7         |0.0        |
|00446675fab526fc7b768e18ed051e3b5e341d5078fd2508c9c03f5258a2389a|3.7         |0.0        |
|00446675fab526fc7b768e18ed051e3b5e341d5078fd2508c9c03f5258a2389a|3.7         |0.0        |
|0148a24b0c6ea3da5f03ac5f516fe030d63fb88d222f1cd417073c7bb7edd71e|1.0         |0.0        |
|02a4d8afc1c0359a3c0d28e3cd55cd8956ba02af055260a50c113b91e91e4573|2.0         |0.0        |
|02a4d8afc1c0359a3c0d28e3cd55cd8956ba02af055260a50c113b91e91e4573|2.0         |0.0        |
|033611b3c479b8d62fb04b2377378f46d8017170ef758ce3f594b04934de7f4d|6.5         |1.0        |
|033611b3c479b8d62fb04b2377378f46d8017170ef758ce3f594b04934de7f4d|6.5         |1

isHeavyUser 라는 컬럼을 만들어서 1과 0으로 라벨링 해주었다

In [29]:
dataArr = d5.randomSplit([0.7, 0.3])
train = dataArr[0]
test = dataArr[1]

In [42]:
# HeavyUser 판단에 사용된 cnt_dt와 mean(cnt_dt) 제외
indexer = StringIndexer(inputCol="acc_id", outputCol="id_code")
assembler = VectorAssembler(inputCols=['wk', 'play_time', 'npc_exp', 'npc_hongmun',
       'quest_exp', 'quest_hongmun', 'item_hongmun', 'game_combat_time',
       'get_money', 'duel_cnt', 'duel_win', 'partybattle_cnt',
       'partybattle_win', 'cnt_enter_inzone_solo', 'cnt_enter_inzone_light',
       'cnt_enter_inzone_skilled', 'cnt_enter_inzone_normal', 'cnt_enter_raid',
       'cnt_enter_raid_light', 'cnt_enter_bam', 'cnt_clear_inzone_solo',
       'cnt_clear_inzone_light', 'cnt_clear_inzone_skilled',
       'cnt_clear_inzone_normal', 'cnt_clear_raid', 'cnt_clear_raid_light',
       'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat',
       'party_chat', 'guild_chat', 'faction_chat', 'cnt_use_buffitem',
       'gathering_cnt', 'making_cnt'], outputCol="features")
dt = DecisionTreeClassifier(labelCol="isHeavyUser", featuresCol="features").setMaxBins(40)

pipeline = Pipeline(stages=[indexer, assembler, dt])
model = pipeline.fit(train)

predict = model.transform(test)
predict.select("probability", "prediction", "isHeavyUser").show(3, False)

+----------------------------------------+----------+-----------+
|probability                             |prediction|isHeavyUser|
+----------------------------------------+----------+-----------+
|[0.032476357664646925,0.967523642335353]|1.0       |1.0        |
|[0.14059775840597757,0.8594022415940225]|1.0       |1.0        |
|[0.14059775840597757,0.8594022415940225]|1.0       |1.0        |
+----------------------------------------+----------+-----------+
only showing top 3 rows



In [43]:
predict.select("probability", "prediction", "isHeavyUser").show(3, False)

+----------------------------------------+----------+-----------+
|probability                             |prediction|isHeavyUser|
+----------------------------------------+----------+-----------+
|[0.032476357664646925,0.967523642335353]|1.0       |1.0        |
|[0.14059775840597757,0.8594022415940225]|1.0       |1.0        |
|[0.14059775840597757,0.8594022415940225]|1.0       |1.0        |
+----------------------------------------+----------+-----------+
only showing top 3 rows



In [44]:
# areaUnderROC, areaUnderPR
evaluator = BinaryClassificationEvaluator(labelCol="isHeavyUser", metricName="areaUnderROC")

In [45]:
print(evaluator.evaluate(predict))

treeModel = model.stages[2]
print("Learned classification tree model:%s" % treeModel.toDebugString)

0.7030415516310541
Learned classification tree model:DecisionTreeClassificationModel (uid=DecisionTreeClassifier_41559485db47bbb05644) of depth 5 with 63 nodes
  If (feature 8 <= -0.0234785876090536)
   If (feature 28 <= -0.188697394931091)
    If (feature 1 <= -0.6564500547355165)
     If (feature 1 <= -0.6589326083971996)
      If (feature 1 <= -0.6593183438267045)
       Predict: 0.0
      Else (feature 1 > -0.6593183438267045)
       Predict: 0.0
     Else (feature 1 > -0.6589326083971996)
      If (feature 0 <= 7.5)
       Predict: 0.0
      Else (feature 0 > 7.5)
       Predict: 0.0
    Else (feature 1 > -0.6564500547355165)
     If (feature 4 <= -0.249941107071387)
      If (feature 1 <= -0.106589145302807)
       Predict: 0.0
      Else (feature 1 > -0.106589145302807)
       Predict: 1.0
     Else (feature 4 > -0.249941107071387)
      If (feature 1 <= 1.78864770767749)
       Predict: 0.0
      Else (feature 1 > 1.78864770767749)
       Predict: 1.0
   Else (feature 28 > -0.1

In [46]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f579116e5c0>>